# CHAPTER 4 LINEAR PROGRAMMING: MODELING EXAMPLES
**Introduction_to_Management_Science**\
**Page 137**

##Parameters Definition
* $I$: The set of weeks needed to be scheduled , \{1,2,3,4,5,6\}
* $CO_i \equiv $ Computers orders for week  $i$, $i\in I$.

##Decision Variables
\begin{align*}
    r_i & \equiv \text{ regular production of computers per week i (i = 1, 2, 3, 4, 5, 6)} \\
    o_i & \equiv \text{ overtime production of computers per week i (i = 1, 2, 3, 4, 5, 6)}\\
    i_j& \equiv \text{extra computers carried over as inventory in week j (j = 1, 2, 3, 4, 5)}
\end{align*}

##Linear Programme
\begin{align*}
    \min \hspace{10pt} & 190  \sum_{i \in I} r_i+260  \sum_{i \in I} o_i +10  \sum_{i \in I} i_i\\
    \text{s.t.} \hspace{7pt}
    & r_i \le 160 & \forall i \in I\\
    & o_i \le 50 & \forall i \in I\\
    & r_i + o_i+ i_{i-1} -i_i =CO_i & \forall i \in I- {1,6}\\
    & r_1 + o_1-i_1 =105\\
    & r_6+ o_6+i_5 =250\\
    & r_i, o_i, i_i\ge 0 & \forall i \in I\\
  \end{align*}


In [ ]:
!pip install cplex
!pip install docplex


     |████████████████████████████████| 43.3 MB 1.2 MB/s 
     |████████████████████████████████| 610 kB 13.9 MB/s 
  Created wheel for docplex: filename=docplex-2.23.222-py3-none-any.whl size=662847 sha256=ce91231a7b7ebda1cc94f40f9fd420c98deea59abd8b83e4e3c522c98e222143
  Stored in directory: /root/.cache/pip/wheels/a7/c9/fb/cee5a89f304e77a39c466e625ac2830434b76eb8384999d116
Successfully built docplex


In [ ]:
# Two index decison variable


In [ ]:
# First impot the Model class from docplex.mp
from docplex.mp.model import Model

In [ ]:
# Create one model instance, with a name
m = Model (name="Multi_Period_Work_Scheduling")

In [ ]:
# Defining the parametrs of the problem statement
#Weeks = ['Week_1','Week_2','Week_3','Week_4','Week_5','Week_6']
Computers_Orders= {1: 105,
                   2: 170,
                   3: 230,
                   4: 180,
                   5: 150,
                   6: 250
                   }
Unit_Regualr_Production_Cost = 190
Unit_Over_Time_Production_Cost = 260
Unit_Holding_Cost = 10
Capacity_Regualr_Time = 160
Capacity_Over_Time = 50
#I_0 = 0
#I_f= 0

#List_Keys= Computers_Orders.keys()
#Computers_Orders.keys().index("Week_1")

In [ ]:
# Defining the decison variabels of the problem
Regular_Production  = m.continuous_var_dict(Computers_Orders.keys(),lb=0,ub=None,name=" Reular Production of Computers Per Week  %s")
Over_Time_Production  = m.continuous_var_dict(Computers_Orders.keys(),lb=0,ub=None,name=" Over_Time Production of Computers Per Week  %s")
#Inventory_Carried_Over= m.continuous_var_dict( for i in Weeks if i != 6,lb=0,ub=None,name=" Extra Computers Carried Over as Inventory in week %s")
Inventory_Carried_Over= m.continuous_var_dict([i for i in range(0,len(Computers_Orders.keys())+1)] ,lb=0,ub=None,name=" Extra Computers Carried Over as Inventory in week %s")



In [ ]:
# Defining the objective function of the problem
Regular_Time_Cost = m.sum(Unit_Regualr_Production_Cost*Regular_Production[i] for i in Computers_Orders.keys())
Over_Time_Cost = m.sum(Unit_Over_Time_Production_Cost*Over_Time_Production [i] for i in Computers_Orders.keys())
Holding_Cost = m.sum(Unit_Holding_Cost*Inventory_Carried_Over [i] for i in range (1,7))
Total_Cost = Regular_Time_Cost+Over_Time_Cost+ Holding_Cost
m.minimize(Total_Cost)
#m.minimize (m.sum((Regular_Production[i]*190)+ (Over_Time_Production [i] *260)  for i in Computers_Orders.keys()) +m.sum(Inventory_Carried_Over [i]*10 for i in range (1,6)))

In [ ]:
# Defining the constraints of the problem
m.add_constraints_(Regular_Production [i]<= Capacity_Regualr_Time for i in Computers_Orders.keys())

m.add_constraints_(Over_Time_Production [i]<= Capacity_Over_Time for i in Computers_Orders.keys())

m.add_constraints_(Regular_Production [i] +Over_Time_Production [i] +Inventory_Carried_Over[i-1]- Inventory_Carried_Over[i] ==Computers_Orders[i] for i in range (2,6) )


# Each two consecutive  out of the following four constraints are the same
m.add_constraint_(Regular_Production [1] +Over_Time_Production [1]  - Inventory_Carried_Over[1] == Computers_Orders[1])

m.add_constraint_(Regular_Production [6] +Over_Time_Production [6]  +  Inventory_Carried_Over[5] == Computers_Orders[6])

#m.add_constraint(Inventory_Carried_Over[0]==I_0)

#m.add_constraint_(Inventory_Carried_Over[len(Computers_Orders.keys())]==I_f)


In [ ]:
# Print the model information
m.print_information()

Model: Multi_Period_Work_Scheduling
 - number of variables: 19
   - binary=0, integer=0, continuous=19
 - number of constraints: 18
   - linear=18
 - parameters: defaults
 - objective: minimize
 - problem type is: LP


In [ ]:
# Solving the model and getting the results
m.solve (log_output =True)

print (m.solve_details.status)

m.print_solution()

Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 12 rows and 2 columns.
Aggregator did 2 substitutions.
Reduced LP has 4 rows, 15 columns, and 18 nonzeros.
Presolve time = 0.01 sec. (0.02 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =         10600.000000
optimal
objective: 216300.000
  " Reular Production of Computers Per Week  1"=160.000
  " Reular Production of Computers Per Week  2"=160.000
  " Reular Production of Computers Per Week  3"=160.000
  " Reular Production of Computers Per Week  4"=160.000
  " Reular Production of Computers Per Week  5"=160.000
  " Reular Production of Computers Per Week  6"=160.000
  " Over_Time Production of Computers Per Week  3"=25.000
  " Over_Time Production of Computers Per Week  4"=20.000
  " Over_Time Production of Computers Per Week  5"=30.000
  " Over_Time Production of Computers Per W